<a href="https://colab.research.google.com/github/leemimi/Programmers/blob/main/%EC%B2%AD%EA%B2%BD%EC%B1%84_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_%EB%AF%B8%EC%A0%951215.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

Dataset Info.

train_input [폴더] - 총 28개 상추 케이스
DAT : 생육 일 (0~27일차)
obs_time : 측정 시간
상추 케이스 별 환경 데이터 (1시간 간격)


train_target [폴더] - 총 28개 상추 케이스
DAT : 생육 일 (1~28일차)
predicted_weight_g : 일별 잎 중량


test_input [폴더] - 총 5개 상추 케이스
DAT : 생육 일 (0~27일차)
obs_time : 측정 시간
상추 케이스 별 환경 데이터 (1시간 간격)


test_target [폴더] - 총 5개 상추 케이스
DAT : 생육 일 (1~28일차)
predicted_weight_g : 일별 예측한 잎 중량
제출을 위한 양식으로 target에 해당되는 predicted_weight_g의 값은 모두 0으로 가려져있습니다.


sample_submission.zip [제출양식]
[정량 평가] 예측 모델 평가를 위한 제출 양식
Test 청경채 케이스 6개에 대한 일별 추론한 결과

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !pip install lightgbm==3.3.2
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory

In [3]:
import random
import pandas as pd
import numpy as np
import os
import glob
import copy

from tqdm.auto import tqdm
import sys
import warnings
warnings.filterwarnings(action='ignore') 


#머신러닝 라이브러리
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import lightgbm as lgb

Seed 고정

In [5]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(seed=42)

In [6]:
base_path = '/content/drive/MyDrive/Colab Notebooks/open'
train_input_list = sorted(glob.glob(base_path+'/train_input/*.csv'))
train_target_list = sorted(glob.glob(base_path+'/train_target/*.csv'))
test_input_list = sorted(glob.glob(base_path+'/test_input/*.csv'))
test_target_list = sorted(glob.glob(base_path+'/test_target/*.csv'))

In [7]:
pd.read_csv(train_input_list[0]).columns, len(pd.read_csv(train_input_list[0]).columns)

(Index(['DAT', 'obs_time', '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량',
        '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
        '일간누적청색광량', '시간당총광량', '일간누적총광량'],
       dtype='object'), 16)

In [8]:
feature_list = ['DAT', 'obs_time', '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량',
        '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
        '일간누적청색광량', '시간당총광량', '일간누적총광량']

베이스라인 코드 분석

In [9]:
df = pd.read_csv(train_input_list[0])
df.head()

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
0,0,00:00,25.300000,81.835000,536.016667,1.407439,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0
1,0,01:00,25.680357,81.264286,528.696429,1.409003,126.0,126.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0
2,0,02:00,25.273333,81.471666,532.833333,1.406913,0.0,126.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0
3,0,03:00,25.355000,81.398334,545.566667,1.406689,126.0,252.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0
4,0,04:00,25.391667,81.483333,558.583333,1.411070,0.0,252.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0


In [10]:
pd.read_csv(train_target_list[0]).head()

,DAT,predicted_weight_g
0,1,0.167719
1,2,0.181787
2,3,0.265921
3,4,0.423650
4,5,0.475272


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DAT       672 non-null    int64  
 1   obs_time  672 non-null    object 
 2   내부온도관측치   672 non-null    float64
 3   내부습도관측치   672 non-null    float64
 4   co2관측치    672 non-null    float64
 5   ec관측치     672 non-null    float64
 6   시간당분무량    672 non-null    float64
 7   일간누적분무량   672 non-null    float64
 8   시간당백색광량   672 non-null    float64
 9   일간누적백색광량  672 non-null    float64
 10  시간당적색광량   672 non-null    float64
 11  일간누적적색광량  672 non-null    float64
 12  시간당청색광량   672 non-null    int64  
 13  일간누적청색광량  672 non-null    int64  
 14  시간당총광량    672 non-null    float64
 15  일간누적총광량   672 non-null    float64
dtypes: float64(12), int64(3), object(1)
memory usage: 84.1+ KB


In [16]:
def merge_data(input_paths, target_paths):
  idx = 0

  temp_df=pd.DataFrame()
  for input_path, target_path in tqdm(zip(input_paths, target_paths)):

    idx +=1

    input_df = pd.read_csv(input_path) # input 
    target_df = pd.read_csv(target_path) # target


    input_df.columns = feature_list


    input_df["obs_time"] = pd.to_datetime(input_df["obs_time"])
    input_df["obs_time"] = input_df["obs_time"].dt.hour

    df = pd.merge(input_df, target_df) # 시간 기준 rate 합쳐줌 
    temp_df = pd.concat([temp_df, df], axis=0)
        
    temp_df = temp_df.reset_index(drop=True)    
    return temp_df


train_df = merge_data(train_input_list, train_target_list)
test_df = merge_data(test_input_list, test_target_list)



0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [20]:
train_df.head()

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,predicted_weight_g
0,1,0,25.806666,81.765000,523.216667,1.390210,126.0,126.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.167719
1,1,1,25.981034,81.329310,531.844828,1.391945,0.0,126.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.167719
2,1,2,25.583334,81.368335,540.633333,1.392577,126.0,252.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.167719
3,1,3,25.476667,81.651667,548.516667,1.394363,0.0,252.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.167719
4,1,4,25.431666,81.986666,555.633333,1.396148,126.0,378.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.167719


In [21]:
train_df.shape , test_df.shape

((648, 17), (648, 17))

In [22]:
fig, axs = plt.subplots(nrows=8, ncols=8, figsize=(21,20))

axs = axs.ravel(order='C')
for i, v in enumerate(train_df["case"].unique()):
    axs[i].plot(train_df["rate"][train_df["case"]==v].reset_index(drop=True))
    axs[i].set_title(f"case_{v}")
plt.tight_layout()
plt.show()

NameError: ignored